# Importing stuff

In [103]:
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd
import datetime as dt
from pathlib import Path
import hvplot.pandas
import plotly.express as px
from bokeh.models.formatters import DatetimeTickFormatter

# Importing API data that will be the crtieria for which zip codes we're looking at.

In [2]:
# Modify this section for your specific computer environment
dotenv_filepath = 'C:/Users/danie/.env'
load_dotenv(dotenv_path=dotenv_filepath)

True

In [3]:
api_key = os.getenv("ZIPCODE_API_KEY")

In [4]:
# Entering in variables, could make this an input feature if we have time
zip_code = 77002
distance = 5
units = "miles"
url = f"http://www.zipcodeapi.com/rest/{api_key}/radius.json/{zip_code}/{distance}/{units}"

In [5]:
response_data = requests.get(url)
data = response_data.json()
print(json.dumps(data, indent=4))

{
    "zip_codes": [
        {
            "zip_code": "77021",
            "distance": 4.071,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77030",
            "distance": 4.082,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77005",
            "distance": 4.396,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77023",
            "distance": 3.714,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77004",
            "distance": 2.211,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77046",
            "distance": 4.39,
            "city": "Houston",
            "state": "TX"
        },
        {
            "zip_code": "77098",
            "distance": 3.418,
            "city": "Houston",
            "state": "T

In [6]:
zip_code = []
city = []

for line in data["zip_codes"]:

    new_zip_code = int(line["zip_code"])
    zip_code.append(new_zip_code)
    
    new_city = line["city"]
    city.append(new_city)

In [7]:
zip_code_df = pd.DataFrame()
zip_code_df['zip_code']  = zip_code
zip_code_df['City']  = city

# Importing function that will be used to filter datafiles

## must set the function = to a new dataframe or it won't save!!

In [8]:
def filter_zip_code(zip_codes_list, df, new_dataframe):
    for zip_code_numbers in zip_codes_list:
        if zip_code_numbers in df:
            data_adding = df.loc[:, zip_code_numbers]
            new_dataframe = new_dataframe.append(data_adding)
    return new_dataframe    

In [9]:
## example
## random_dataframe = pd.DataFrame()

## random_dataframe = filter_zip_code(zip_codes_list, df, random_dataframe)

## random_dataframe should be a dataframe with all the zip codes we're looking at

# Importing tax data and creating the dataframes

In [10]:
# importing the data
clean_path = Path("../resources/Vu/Tax_Data/cleaned_tax_FINALs.csv")

clean_tax = pd.read_csv(clean_path)

clean_tax = clean_tax.rename({'Zip Code': 'zip_code'}, axis=1)

clean_tax = clean_tax.astype({'zip_code': 'int'})

clean_tax.drop(columns = "Size of adjusted gross income", inplace = True)

multi_index_tax = clean_tax.groupby(["zip_code", "date"]).sum()


# creating a new dataframe for each of the columns
number_of_returns_df = pd.pivot_table(multi_index_tax, index = "date", values = "Number of returns", columns = "zip_code")

taxable_income_df = pd.pivot_table(multi_index_tax, index = "date", values = "Taxable income", columns = "zip_code")

income_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Income tax", columns = "zip_code")

mortgage_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Mortgage", columns = "zip_code")

charity_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Charity", columns = "zip_code")

real_estates_tax_df = pd.pivot_table(multi_index_tax, index = "date", values = "Real Estates", columns = "zip_code")



# Setting index to datetime and sorting index(date)
number_of_returns_df.index = pd.to_datetime(number_of_returns_df.index)
number_of_returns_df.sort_index(inplace=True)

taxable_income_df.index = pd.to_datetime(taxable_income_df.index)
taxable_income_df.sort_index(inplace=True)

income_tax_df.index = pd.to_datetime(income_tax_df.index)
income_tax_df.sort_index(inplace=True)

mortgage_tax_df.index = pd.to_datetime(mortgage_tax_df.index)
mortgage_tax_df.sort_index(inplace=True)

charity_tax_df.index = pd.to_datetime(charity_tax_df.index)
charity_tax_df.sort_index(inplace=True)

real_estates_tax_df.index = pd.to_datetime(real_estates_tax_df.index)
real_estates_tax_df.sort_index(inplace=True)

# Importing permit data and creating the dataframes

In [11]:
permits_filepath = Path("../resources/dan/Permit_Data/permit_data.csv")

permits_df = pd.read_csv(permits_filepath)

permits_df.dropna(inplace=True)

permits_df = permits_df.astype({'zip_code': 'int64'})

permits_df.set_index(pd.to_datetime(permits_df['date'], format='%Y%m', infer_datetime_format=True), inplace=True)

permits_df.drop('date', axis=1, inplace=True)

permit_counts = permits_df.groupby(["zip_code", "date"]).sum()

permits_by_zip = pd.pivot_table(permit_counts, index = "date", values = "count", columns = "zip_code")

#permits_by_zip.index = permits_by_zip.index.to_period('M')

permits_by_zip = permits_by_zip.groupby(permits_by_zip.index).sum()

permits_by_zip.fillna(0, inplace=True)

permits_by_zip = permits_by_zip.astype(int)


# Importing Texas Pricing Data and creating the dataframes

In [12]:
csvpath = Path("../resources/Jack/Texas_Pricing_Data.csv")

pricing_data_df = pd.read_csv(csvpath)

pricing_data_df.set_index(pd.to_datetime(pricing_data_df['month_date_yyyymm'], format='%Y%m', infer_datetime_format=True), inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.index.rename('date', inplace=True)

pricing_data_df.sort_values(by=['postal_code','date'], inplace=True)

pricing_data_df['pct_price_change'] = 0

pricing_data_df['pct_price_sqft_change'] = 0

pricing_data_df['pct_price_sqft_change'] = pricing_data_df['median_listing_price_per_square_feet'].pct_change()

pricing_data_df.index.rename('date', inplace=True)

median_list_sqft_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'median_listing_price_per_square_feet')

median_list_sqft_df_pct_change = median_list_sqft_df.pct_change()

active_listings_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'active_listing_count')

new_listing_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'new_listing_count')

average_listing_price_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'average_listing_price')

pending_ratio_df = pd.pivot_table(pricing_data_df, index='date', columns = 'postal_code', values = 'pending_ratio')



# Creating filtered datasets of the zipcodes we want to look at
# Also filling in all NaN vaules to not delete rows, we can choose to delete if we don't want that zip portrayed.

### starting with the Texas Pricing Data set

In [13]:
# filtered_median_list_sqft_df = pd.DataFrame()
# filtered_median_list_sqft_df = filter_zip_code(zip_code, median_list_sqft_df, filtered_median_list_sqft_df)
# filtered_median_list_sqft_df.head()

# list(filtered_median_list_sqft_df.index.values)

In [14]:

# # create list of zips we want to use based on results from above filtering
# zip_code_master_list = []
# for zip_code in list(filtered_median_list_sqft_df.index.values):
#     zip_code_master_list.append(zip_code)
# print(zip_code_master_list)


In [26]:
filtered_median_list_sqft_df = pd.DataFrame()
filtered_median_list_sqft_df = filter_zip_code(zip_code, median_list_sqft_df, filtered_median_list_sqft_df)

# # Use above filtered list columns (zips) to create a list of zips in radius based on results from above filtering. for simplified reference in later dataframes
zip_code_master_list = []
for zc in list(filtered_median_list_sqft_df.index.values):
    zip_code_master_list.append(zc)


filtered_median_list_sqft_df.fillna(0, inplace = True)
filtered_median_list_sqft_df = filtered_median_list_sqft_df.transpose(copy=True)
filtered_median_list_sqft_df.index = filtered_median_list_sqft_df.index.to_period('M')

filtered_median_list_sqft_df_pct_change = pd.DataFrame()
filtered_median_list_sqft_df_pct_change = filter_zip_code(zip_code, median_list_sqft_df_pct_change, filtered_median_list_sqft_df_pct_change)
filtered_median_list_sqft_df_pct_change.fillna(0, inplace = True)
filtered_median_list_sqft_df_pct_change = filtered_median_list_sqft_df_pct_change.transpose(copy=True)
filtered_median_list_sqft_df_pct_change.index = filtered_median_list_sqft_df_pct_change.index.to_period('M')

filtered_active_listings_df = pd.DataFrame()
filtered_active_listings_df = filter_zip_code(zip_code, active_listings_df, filtered_active_listings_df)
filtered_active_listings_df.fillna(0, inplace = True)
filtered_active_listings_df = filtered_active_listings_df.transpose(copy=True)
filtered_active_listings_df.index = filtered_active_listings_df.index.to_period('M')
filtered_active_listings_df = filtered_active_listings_df.astype(int)

filtered_new_listing_df = pd.DataFrame()
filtered_new_listing_df = filter_zip_code(zip_code, new_listing_df, filtered_new_listing_df)
filtered_new_listing_df.fillna(0, inplace = True)
filtered_new_listing_df = filtered_new_listing_df.transpose(copy=True)
filtered_new_listing_df.index = filtered_new_listing_df.index.to_period('M')
filtered_new_listing_df = filtered_new_listing_df.astype(int)

filtered_average_listing_price_df = pd.DataFrame()
filtered_average_listing_price_df = filter_zip_code(zip_code, average_listing_price_df, filtered_average_listing_price_df)
filtered_average_listing_price_df.fillna(0, inplace = True)
filtered_average_listing_price_df = filtered_average_listing_price_df.transpose(copy=True)
filtered_average_listing_price_df.index = filtered_average_listing_price_df.index.to_period('M')

filtered_pending_ratio_df = pd.DataFrame()
filtered_pending_ratio_df = filter_zip_code(zip_code, pending_ratio_df, filtered_pending_ratio_df)
filtered_pending_ratio_df.fillna(0, inplace = True)
filtered_pending_ratio_df = filtered_pending_ratio_df.transpose(copy=True)
filtered_pending_ratio_df.index = filtered_pending_ratio_df.index.to_period('M')

filtered_pricing_data_df = pricing_data_df[pricing_data_df['postal_code'].isin(zip_code_master_list)]

### now the permits data set

In [27]:
filtered_permits_by_zip = pd.DataFrame()
filtered_permits_by_zip = filter_zip_code(zip_code, permits_by_zip, filtered_permits_by_zip)
filtered_permits_by_zip.fillna(0, inplace = True)
filtered_permits_by_zip = filtered_permits_by_zip.transpose(copy=True)
filtered_permits_by_zip.index = filtered_permits_by_zip.index.to_period('M')
filtered_permits_by_zip = filtered_permits_by_zip.astype(int)

filtered_permits_df = permits_df[permits_df['zip_code'].isin(zip_code_master_list)]

### finally the tax data set

In [31]:
filtered_number_of_returns_df = pd.DataFrame()
filtered_number_of_returns_df = filter_zip_code(zip_code, number_of_returns_df, filtered_number_of_returns_df)
filtered_number_of_returns_df.fillna(0, inplace = True)
filtered_number_of_returns_df = filtered_number_of_returns_df.transpose(copy=True)
filtered_number_of_returns_df.index = filtered_number_of_returns_df.index.to_period('M')
filtered_number_of_returns_df = filtered_number_of_returns_df.astype(int)

filtered_taxable_income_df = pd.DataFrame()
filtered_taxable_income_df = filter_zip_code(zip_code, taxable_income_df, filtered_taxable_income_df)
filtered_taxable_income_df.fillna(0, inplace = True)
filtered_taxable_income_df = filtered_taxable_income_df.transpose(copy=True)
filtered_taxable_income_df.index = filtered_taxable_income_df.index.to_period('M')

filtered_income_tax_df = pd.DataFrame()
filtered_income_tax_df = filter_zip_code(zip_code, income_tax_df, filtered_income_tax_df)
filtered_income_tax_df.fillna(0, inplace = True)
filtered_income_tax_df = filtered_income_tax_df.transpose(copy=True)
filtered_income_tax_df.index = filtered_income_tax_df.index.to_period('M')

filtered_mortgage_tax_df = pd.DataFrame()
filtered_mortgage_tax_df = filter_zip_code(zip_code, mortgage_tax_df, filtered_mortgage_tax_df)
filtered_mortgage_tax_df.fillna(0, inplace = True)
filtered_mortgage_tax_df = filtered_mortgage_tax_df.transpose(copy=True)
filtered_mortgage_tax_df.index = filtered_mortgage_tax_df.index.to_period('M')

filtered_charity_tax_df = pd.DataFrame()
filtered_charity_tax_df = filter_zip_code(zip_code, charity_tax_df, filtered_charity_tax_df)
filtered_charity_tax_df.fillna(0, inplace = True)
filtered_charity_tax_df = filtered_charity_tax_df.transpose(copy=True)
filtered_charity_tax_df.index = filtered_charity_tax_df.index.to_period('M')

filtered_real_estates_tax_df = pd.DataFrame()
filtered_real_estates_tax_df = filter_zip_code(zip_code, real_estates_tax_df, filtered_real_estates_tax_df)
filtered_real_estates_tax_df.fillna(0, inplace = True)
filtered_real_estates_tax_df.fillna(0, inplace = True)
filtered_real_estates_tax_df = filtered_real_estates_tax_df.transpose(copy=True)
filtered_real_estates_tax_df.index = filtered_real_estates_tax_df.index.to_period('M')

filtered_clean_tax = clean_tax[clean_tax['zip_code'].isin(zip_code_master_list)]

In [21]:
filtered_pricing_data_df = pricing_data_df[pricing_data_df['postal_code'].isin(zip_code_master_list)]



,month_date_yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_feet,median_square_feet,average_listing_price,total_listing_count,pending_ratio,pct_price_change,pct_price_sqft_change
date,,,,,,,,,,,,,,,,,
2016-07-01,201607,77002,"houston, tx",312050.0,52,88.0,12,0,8,15,253.271422,1266.0,499723,67,0.2885,0,NaN
2016-08-01,201608,77002,"houston, tx",349050.0,54,89.5,12,0,12,20,250.615461,1365.5,505122,74,0.3704,0,-0.010487
2016-09-01,201609,77002,"houston, tx",339050.0,57,76.5,12,0,8,12,248.215391,1375.5,476139,69,0.2105,0,-0.009577
2016-10-01,201610,77002,"houston, tx",326050.0,55,80.0,8,0,4,8,241.769547,1338.0,439559,63,0.1455,0,-0.025969
2016-11-01,201611,77002,"houston, tx",329050.0,53,83.0,16,0,4,8,245.061046,1276.5,425851,61,0.1509,0,0.013614


# Analysis Section

In [32]:
# ### dataframe variable reference
# zip_code_master_list
#
# #### Pricing dataframes
# filtered_pricing_data_df
# 
# filtered_median_list_sqft_df  
# filtered_median_list_sqft_df_pct_change  
# filtered_active_listings_df  
# filtered_new_listing_df  
# filtered_average_listing_price_df  
# filtered_pending_ratio_df  


# #### Permits dataframes
# filtered_permits_df
#
# filtered_permits_by_zip  


# #### Tax dataframes
# filtered_clean_tax
# 
# filtered_number_of_returns_df  
# filtered_taxable_income_df  
# filtered_income_tax_df  
# filtered_mortgage_tax_df  
# filtered_charity_tax_df  
# filtered_real_estates_tax_df  

#### Overall listings per month/year

In [117]:
# Calculate the mean number of housing units per year (hint: use groupby) 
# YOUR CODE HERE!
mean_active_listings_by_date = filtered_pricing_data_df.groupby('date')['active_listing_count'].mean()
mean_active_listings_by_date

# calculate min, max, and std of mean_housing_units_per_year 
min_mean_active_listings = mean_active_listings_by_date.min()
max_mean_active_listings = mean_active_listings_by_date.max()
std_mean_active_listings = mean_active_listings_by_date.std()

# Set ylim min and ylim max values based on min/max/std of housing units above for better scale of housing units
ylim_min = min_mean_active_listings - std_mean_active_listings
ylim_max = max_mean_active_listings + std_mean_active_listings

x_formatter = DatetimeTickFormatter(months='%Y%m')

hyplot_active_listings = mean_active_listings_by_date.hvplot.bar(x='date',y='active_listing_count',rot=90).opts(
    title="Overall Active Listings per Month - Entire Area",
    height=400,
    xlabel='Date',
    ylabel='Active Listing',
    #xticks=25,
    #yformatter="%.0f",
    #xformatter=x_formatter,
    ylim=(ylim_min, ylim_max)
)

hyplot_active_listings

:Bars   [date]   (active_listing_count)

#### Overall mean price per square foot / month-year

In [40]:
filtered_pricing_data_df.head()

,month_date_yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_feet,median_square_feet,average_listing_price,total_listing_count,pending_ratio,pct_price_change,pct_price_sqft_change
date,,,,,,,,,,,,,,,,,
2016-07-01,201607,77002,"houston, tx",312050.0,52,88.0,12,0,8,15,253.271422,1266.0,499723,67,0.2885,0,NaN
2016-08-01,201608,77002,"houston, tx",349050.0,54,89.5,12,0,12,20,250.615461,1365.5,505122,74,0.3704,0,-0.010487
2016-09-01,201609,77002,"houston, tx",339050.0,57,76.5,12,0,8,12,248.215391,1375.5,476139,69,0.2105,0,-0.009577
2016-10-01,201610,77002,"houston, tx",326050.0,55,80.0,8,0,4,8,241.769547,1338.0,439559,63,0.1455,0,-0.025969
2016-11-01,201611,77002,"houston, tx",329050.0,53,83.0,16,0,4,8,245.061046,1276.5,425851,61,0.1509,0,0.013614


In [95]:
avg_prices = filtered_pricing_data_df.groupby('date')['median_listing_price','median_listing_price_per_square_feet'].mean()
# avg_price_sqft_by_date = filtered_pricing_data_df.groupby('date').mean()

avg_prices.head()


,median_listing_price,median_listing_price_per_square_feet
date,,
2016-07-01,493767.647059,215.785661
2016-08-01,480047.029412,214.186900
2016-09-01,474164.705882,212.393183
2016-10-01,471561.764706,211.325488
2016-11-01,473086.441176,210.938234


In [118]:

hyplot_avg_price_sqft_by_date = avg_prices['median_listing_price_per_square_feet'].hvplot.line(x='date',y='median_listing_price_per_square_feet',rot=90).opts(
    title="Average Price/SqFt - Entire Area",
    height=400,
    xlabel='Date',
    ylabel='Avg Price/SqFt',
    xticks=25,
    #yformatter="%.0f",
    #xformatter=x_formatter,
)

hyplot_avg_price_sqft_by_date


:Curve   [date]   (median_listing_price_per_square_feet)

#### Overall mean sales prices / month-year

In [123]:
hyplot_avg_listing_price = avg_prices['median_listing_price'].hvplot.bar(x='date',y='median_listing_price',rot=90).opts(
    title="Average Listing Prices - Entire Area",
    height=400,
    xlabel='Date',
    ylabel='Avg Listing Prices',
    #xticks=25,
    yformatter="%.0f",
    #xformatter=x_formatter,
)

hyplot_avg_listing_price

:Bars   [date]   (median_listing_price)

#### Prices per Area - drop down selector

In [157]:
filtered_pricing_data_df.head()

,month_date_yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_feet,median_square_feet,average_listing_price,total_listing_count,pending_ratio,pct_price_change,pct_price_sqft_change
date,,,,,,,,,,,,,,,,,
2016-07-01,201607,77002,"houston, tx",312050.0,52,88.0,12,0,8,15,253.271422,1266.0,499723,67,0.2885,0,NaN
2016-08-01,201608,77002,"houston, tx",349050.0,54,89.5,12,0,12,20,250.615461,1365.5,505122,74,0.3704,0,-0.010487
2016-09-01,201609,77002,"houston, tx",339050.0,57,76.5,12,0,8,12,248.215391,1375.5,476139,69,0.2105,0,-0.009577
2016-10-01,201610,77002,"houston, tx",326050.0,55,80.0,8,0,4,8,241.769547,1338.0,439559,63,0.1455,0,-0.025969
2016-11-01,201611,77002,"houston, tx",329050.0,53,83.0,16,0,4,8,245.061046,1276.5,425851,61,0.1509,0,0.013614


In [158]:
#avg_sfo_data_grpby_year_nbhd = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
plot_df = filtered_pricing_data_df.groupby(['date','postal_code']).mean().reset_index()
plot_df.head()

plot_df.hvplot.line(x='date',y='median_listing_price_per_square_feet', groupby=['postal_code']).opts(
    title='Median Prices/SqFt per Zip',
    height=400,
    xlabel='Date',
    ylabel='Median Price/SqFt'
)




:DynamicMap   [postal_code]
   :Curve   [date]   (median_listing_price_per_square_feet)